In [ ]:
%pip install PyPDF2
%pip install -U PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 105.0 MB/s eta 0:00:00


# Import libraries

In [ ]:
import re, pathlib, os, fitz
from pathlib import Path
from PyPDF2 import PdfReader

# Extracting constitution law

In [ ]:
# Path to the PDF file
pdf_path = "/content/CONSTITUTION OF THE REPUBLIC OF KOREA.pdf"

# Read the PDF
reader = PdfReader(pdf_path)
full_text = ""

# Collect text from all pages (remove page numbers and word breaks)
for page in reader.pages:
    text = page.extract_text()
    if text:
        # Remove page numbers like "Page 1", "Page 2", etc.
        text = re.sub(r'\bPage\s*\d+\b', '', text)  # \s* matches any number of spaces (0 or more)
        # Remove lines that consist only of numbers (likely page headers/footers)
        text = re.sub(r'^\s*\d+\s*$', '', text, flags=re.MULTILINE)
        # Remove line breaks within paragraphs, while preserving double line breaks (new paragraphs)
        text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
        full_text += text + "\n"

# Split text by "Article" headers (e.g., Article 1, Article 2, ...)
# The (?=...) is a lookahead to keep the "Article N" line at the start of each chunk
article_chunks = re.split(r'(?=Article \d+)', full_text)

# Path to save the result
output_path = "/content/result.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for chunk in article_chunks:
        chunk = chunk.strip()
        if chunk:
            f.write(chunk + "\n\n---\n\n")

# Extracting Pharmaceutical laws

In [ ]:
# ────────────────────────────────
# 1. Just modify the path to your PDF
# ────────────────────────────────
PDF_PATH = "/content/PHARMACEUTICAL AFFAIRS ACT.pdf"   # ★ Set to your desired location

# ────────────────────────────────
# 2. Output will be saved in the same folder as the PDF
# ────────────────────────────────
BASE_DIR   = pathlib.Path(PDF_PATH).expanduser().resolve().parent
# .expanduser(): expands ~ to full home directory path
# .resolve(): resolves relative paths and symbolic links to absolute paths
# .parent: gets the parent directory of the file (i.e., the folder containing the PDF)

CLEAN_TXT  = BASE_DIR / "pharma_cleaned.txt"
BYCHAP_TXT = BASE_DIR / "pharma_by_chapter.txt"

# ────────────────────────────────
# 3. Common text-cleaning function
def clean_text(txt: str) -> str:
    # txt: str → str means this function takes and returns a string (type hints)

    # 1) Patterns to remove in bulk
    # .*? means match any character 0 or more times, non-greedy (as short as possible)
    # \] and \. are escaped to match literal characters
    patterns = [
        r'\bPage\s*\d+\b',
        r'법제처.*?국가법령정보센터',
        r'www\.law\.go\.kr',
        r'\d{2,4}-\d{3,4}-\d{4}',  # \d{2,4}: 2 to 4 digit numbers
        r'PHARMACEUTICAL AFFAIRS ACT',
        r'\[Enforcement Date.*?\]',
        r'\[Act No\..*?\]',
        r'\s*터\s*「\s*」\s*',     # The pattern '터 「」' plus any whitespace or line breaks
    ]

    # Use re.sub to replace all matched patterns with a space (case-insensitive)
    txt = re.sub("|".join(patterns), " ", txt, flags=re.I)
    # Example: re.sub("abc|123|def", " ", txt)

    # 2) Remove isolated Korean characters (e.g., leftover '터')
    txt = re.sub(r'[가-힣]', '', txt)

    # 3) Merge line breaks that appear within lines (not paragraph breaks)
    txt = re.sub(r'(?<!\n)\n(?!\n)', ' ', txt)

    # 4) Remove any remaining empty brackets 「」
    txt = re.sub(r'\s*「\s*」\s*', ' ', txt)

    # 5) Remove any header text before the first CHAPTER
    m = re.search(r'CHAPTER\s+[IVXLC]+\b', txt)  # \s+ means one or more whitespace characters
    if m:
        txt = txt[m.start():]  # Trim text before CHAPTER

    # 6) Convert 3 or more newlines into exactly 2
    txt = re.sub(r'\n{3,}', '\n\n', txt)

    # 7) Collapse multiple spaces into one
    txt = re.sub(r' {2,}', ' ', txt)  # ' {2,}' = 2 or more consecutive spaces

    return txt.strip()  # Remove leading/trailing whitespace

# ────────────────────────────────
# 4. Extract text → clean it
# ────────────────────────────────
if not CLEAN_TXT.exists():
    doc = fitz.open(PDF_PATH)  # Open PDF with PyMuPDF; pages accessible as doc[n]
    raw_text = "\n".join(page.get_text("text") for page in doc)
    # Joins all page texts into one string, separated by line breaks
    CLEAN_TXT.write_text(clean_text(raw_text), encoding="utf-8")

cleaned = CLEAN_TXT.read_text(encoding="utf-8")

# ────────────────────────────────
# 5. Save by CHAPTER block
# ────────────────────────────────
with BYCHAP_TXT.open("w", encoding="utf-8") as f:
    for block in re.split(r'(?=CHAPTER\s+[IVXLC]+\b)', cleaned):
        # (?=...) is a lookahead, so CHAPTER stays at the start of each block
        block = block.strip()
        if not block:
            continue  # Skip if block is empty
        m = re.match(r'CHAPTER\s+[IVXLC]+\b[^\n]*', block)
        # Match full CHAPTER title on the first line (until newline)
        if not m:
            continue
        chap_title = m.group()  # Extract matched CHAPTER title
        body = block[m.end():].lstrip()  # Get the rest of the block, remove left whitespace
        f.write(chap_title + "\n\n")
        f.write(body + "\n\n")
        f.write("=" * 80 + "\n\n")

# Example structure:
# blocks = [
#   '',  # Empty string before CHAPTER I
#   'CHAPTER I GENERAL PROVISIONS\nArticle 1 (Purpose)\n...',
#   'CHAPTER II APPROVAL\nArticle 5 (Approval of drugs)\n...'
# ]

print(f"Complete \n- cleaned → {CLEAN_TXT}\n- by-chapter → {BYCHAP_TXT}")


Complete 
- cleaned → /content/pharma_cleaned.txt
- by-chapter → /content/pharma_by_chapter.txt


Positive Lookahead (?=...): There must be ... after the current position

Negative Lookahead (?!...): There must not be ... after the current position

Positive Lookbehind (?<=...): There must be ... before the current position

Negative Lookbehind (?<!...): There must not be ... before the current position

